In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import subprocess
import pdb
import time
import datetime
import math
import copy
import random
import _pickle as cPickle
from collections import defaultdict
import matplotlib.pyplot as plt

%matplotlib inline
# %matplotlib nbagg

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '1', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/topic_vae', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 1000, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 3000, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
# flags.DEFINE_string('opt', 'Adam', 'optimizer')
flags.DEFINE_float('lr', 0.01, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 20, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
def get_batches(instances, batch_size, iterator=False):
    iter_instances = iter(instances)
    n_batch = len(instances)//batch_size
    
    batches = [(i_batch, [next(iter_instances) for i_doc in range(batch_size)]) for i_batch in range(n_batch)]
    
    if iterator: batches = iter(batches)
    return batches

train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

In [8]:
def debug_shape(variables, model):
    sample_batch = dev_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    

#  model 

## doubly rnn

In [9]:
class DoublyRNNCell:
    def __init__(self, dim_hidden, output_layer=None):
        self.dim_hidden = dim_hidden
        
        self.ancestral_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='ancestral')
        self.fraternal_layer=tf.layers.Dense(units=dim_hidden, activation=tf.nn.tanh, name='fraternal')
        self.hidden_layer = tf.layers.Dense(units=dim_hidden, name='hidden')
        
        self.output_layer=output_layer
        
    def __call__(self, state_ancestral, state_fraternal, reuse=True):
        with tf.variable_scope('input', reuse=reuse):
            state_ancestral = self.ancestral_layer(state_ancestral)
            state_fraternal = self.fraternal_layer(state_fraternal)

        with tf.variable_scope('output', reuse=reuse):
            state_hidden = self.hidden_layer(state_ancestral + state_fraternal)
            if self.output_layer is not None: 
                output = self.output_layer(state_hidden)
            else:
                output = state_hidden
            
        return output, state_hidden
    
    def get_initial_state(self, name):
        initial_state = tf.get_variable(name, [1, self.dim_hidden], dtype=tf.float32)
        return initial_state
    
    def get_zero_state(self, name):
        zero_state = tf.zeros([1, self.dim_hidden], dtype=tf.float32, name=name)
        return zero_state

In [10]:
def doubly_rnn(dim_hidden, tree_idxs, initial_state_parent=None, initial_state_sibling=None, output_layer=None, name=''):
    outputs, states_parent = {}, {}
    
    with tf.variable_scope(name, reuse=False):
        doubly_rnn_cell = DoublyRNNCell(dim_hidden, output_layer)

        if initial_state_parent is None: 
            initial_state_parent = doubly_rnn_cell.get_initial_state('init_state_parent')
#             initial_state_parent = doubly_rnn_cell.get_zero_state('init_state_parent')
        if initial_state_sibling is None: 
#             initial_state_sibling = doubly_rnn_cell.get_initial_state('init_state_sibling')
            initial_state_sibling = doubly_rnn_cell.get_zero_state('init_state_sibling')
        output, state_sibling = doubly_rnn_cell(initial_state_parent, initial_state_sibling, reuse=False)
        outputs[0], states_parent[0] = output, state_sibling

        for parent_idx, child_idxs in tree_idxs.items():
            state_parent = states_parent[parent_idx]
            state_sibling = initial_state_sibling
            for child_idx in child_idxs:
                output, state_sibling = doubly_rnn_cell(state_parent, state_sibling)
                outputs[child_idx], states_parent[child_idx] = output, state_sibling

    return outputs, states_parent

## nCRP model

In [11]:
class Model():
    def __init__(self, config, tree_idxs):
        def get_depth(parent_idx=0, tree_depth=None, depth=1):
            if tree_depth is None: tree_depth={0: depth}

            child_idxs = tree_idxs[parent_idx]
            depth +=1
            for child_idx in child_idxs:
                tree_depth[child_idx] = depth
                if child_idx in tree_idxs: get_depth(child_idx, tree_depth, depth)
            return tree_depth
        
        self.config = config
        
        self.t_variables = {}
        
        self.tree_idxs = tree_idxs
        self.topic_idxs = [0] + [idx for child_idxs in tree_idxs.values() for idx in child_idxs]
        self.child_to_parent_idxs = {child_idx: parent_idx for parent_idx, child_idxs in self.tree_idxs.items() for child_idx in child_idxs}
        self.tree_depth = get_depth()
        self.n_depth = max(self.tree_depth.values())
        
        self.build()
        
    def build(self):
        def nCRP(tree_sticks_topic):
            tree_prob_topic = {}
            tree_prob_leaf = {}
            # calculate topic probability and save
            tree_prob_topic[0] = 1.

            for parent_idx, child_idxs in self.tree_idxs.items():
                rest_prob_topic = tree_prob_topic[parent_idx]
                for child_idx in child_idxs:
                    stick_topic = tree_sticks_topic[child_idx]
                    if child_idx == child_idxs[-1]:
                        prob_topic = rest_prob_topic * 1.
                    else:
                        prob_topic = rest_prob_topic * stick_topic

                    if not child_idx in self.tree_idxs: # leaf childs
                        tree_prob_leaf[child_idx] = prob_topic
                    else:
                        tree_prob_topic[child_idx] = prob_topic

                    rest_prob_topic -= prob_topic
            return tree_prob_leaf

        def get_prob_topic(tree_prob_leaf, prob_depth):
            def get_ancestor_idxs(leaf_idx, ancestor_idxs = None):
                if ancestor_idxs is None: ancestor_idxs = [leaf_idx]
                parent_idx = self.child_to_parent_idxs[leaf_idx]
                ancestor_idxs += [parent_idx]
                if parent_idx in self.child_to_parent_idxs: get_ancestor_idxs(parent_idx, ancestor_idxs)
                return ancestor_idxs[::-1]
            
            tree_prob_topic = defaultdict(float)
            leaf_ancestor_idxs = {leaf_idx: get_ancestor_idxs(leaf_idx) for leaf_idx in tree_prob_leaf}
            for leaf_idx, ancestor_idxs in leaf_ancestor_idxs.items():
                prob_leaf = tree_prob_leaf[leaf_idx]
                for i, ancestor_idx in enumerate(ancestor_idxs):
                    prob_ancestor = prob_leaf * tf.expand_dims(prob_depth[:, i], -1)
                    tree_prob_topic[ancestor_idx] += prob_ancestor
            prob_topic = tf.concat([tree_prob_topic[topic_idx] for topic_idx in self.topic_idxs], -1)
            return prob_topic
        
        def get_tree_topic_bow(tree_topic_embeddings):
            def softmax_with_temperature(logits, axis=None, name=None, temperature=1.):
                if axis is None:
                    axis = -1
                return tf.exp(logits / temperature) / tf.reduce_sum(tf.exp(logits / temperature), axis=axis)

            tree_topic_bow = {}
            for topic_idx, depth in self.tree_depth.items():
                topic_embedding = tree_topic_embeddings[topic_idx]
                temperature = tf.constant(10 ** (1./depth), dtype=tf.float32)
                logits = tf.matmul(topic_embedding, bow_embeddings, transpose_b=True)
                tree_topic_bow[topic_idx] = softmax_with_temperature(logits, axis=-1, temperature=temperature)
            return tree_topic_bow

        def get_tree_mask_reg():
            def get_descendant_idxs(parent_idx, descendant_idxs = None):
                if descendant_idxs is None: descendant_idxs = []

                child_idxs = self.tree_idxs[parent_idx]
                descendant_idxs += child_idxs
                for child_idx in child_idxs:
                    if child_idx in self.tree_idxs: get_descendant_idxs(child_idx, descendant_idxs)
                return descendant_idxs

            tree_mask_reg = np.ones([len(self.topic_idxs), len(self.topic_idxs)], dtype=np.float32)
            parent_to_descendant_idxs = {parent_idx: get_descendant_idxs(parent_idx) for parent_idx in self.tree_idxs}

            for parent_idx, descendant_idxs in parent_to_descendant_idxs.items():
                for descendant_idx in descendant_idxs:
                    parent_index = self.topic_idxs.index(parent_idx)
                    descendant_index = self.topic_idxs.index(descendant_idx)
                    tree_mask_reg[parent_index, descendant_index] = tree_mask_reg[descendant_index, parent_index] = 0.
            return tree_mask_reg
       
        # -------------- Build Model --------------
        tf.reset_default_graph()
        
        self.t_variables['bow'] = tf.placeholder(tf.float32, [None, self.config.dim_bow])
        self.t_variables['keep_prob'] = tf.placeholder(tf.float32)
        
        # encode bow
        with tf.variable_scope('topic/enc', reuse=False):
            hidden_bow_ = tf.layers.Dense(units=self.config.dim_hidden_bow, activation=tf.nn.tanh, name='hidden_bow')(self.t_variables['bow'])
            hidden_bow = tf.layers.Dropout(self.t_variables['keep_prob'])(hidden_bow_)
            means_bow = tf.layers.Dense(units=self.config.dim_latent_bow, name='mean_bow')(hidden_bow)
            logvars_bow = tf.layers.Dense(units=self.config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_bow')(hidden_bow)
            latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors
            prob_layer = lambda h: tf.nn.sigmoid(tf.matmul(latents_bow, h, transpose_b=True))

            tree_sticks_topic, tree_states_sticks_topic = doubly_rnn(self.config.dim_latent_bow, self.tree_idxs, output_layer=prob_layer, name='sticks_topic')
            tree_prob_leaf = nCRP(tree_sticks_topic)
            self.tree_prob_leaf = tree_prob_leaf
            prob_depth = tf.layers.Dense(units=self.n_depth, activation=tf.nn.softmax, name='prob_depth')(latents_bow) # inference of topic probabilities
            self.prob_depth = prob_depth

            prob_topic = get_prob_topic(tree_prob_leaf, prob_depth)
            self.prob_topic = prob_topic

        # decode bow
        with tf.variable_scope('shared', reuse=False):
            embeddings = tf.get_variable('emb', [self.config.n_vocab, self.config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

        bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

        with tf.variable_scope('topic/dec', reuse=False):
        #     tree_topic_embeddings, tree_states_topic_embeddings = doubly_rnn(self.config.dim_emb, self.tree_idxs, name='emb_topic')
            emb_layer = lambda h: tf.layers.Dense(units=self.config.dim_emb, name='output')(tf.nn.tanh(h))
            tree_topic_embeddings, tree_states_topic_embeddings = doubly_rnn(self.config.dim_emb, self.tree_idxs, output_layer=emb_layer, name='emb_topic')
#             topic_embeddings = tf.get_variable('topic_emb', [len(self.topic_idxs), self.config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics
#             tree_topic_embeddings = {topic_idx: tf.expand_dims(topic_embeddings[self.topic_idxs.index(topic_idx)], 0) for topic_idx in self.topic_idxs}

            tree_topic_bow = get_tree_topic_bow(tree_topic_embeddings) # bow vectors for each topic

            topic_bow = tf.concat([tree_topic_bow[topic_idx] for topic_idx in self.topic_idxs], 0)
            self.topic_bow = topic_bow
            logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution            
            self.logits_bow = logits_bow
            
        # define losses
        self.topic_losses_recon = -tf.reduce_sum(tf.multiply(self.t_variables['bow'], logits_bow), 1)
        self.topic_loss_recon = tf.reduce_mean(self.topic_losses_recon) # negative log likelihood of each words

        self.topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

        topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
        self.topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
        self.tree_mask_reg = get_tree_mask_reg()
        self.topic_losses_reg = tf.square(self.topic_dots - tf.eye(len(self.topic_idxs))) * self.tree_mask_reg
        self.topic_loss_reg = tf.reduce_sum(self.topic_losses_reg) / tf.reduce_sum(self.tree_mask_reg)

        self.global_step = tf.Variable(0, name='global_step',trainable=False)

        self.loss = self.topic_loss_recon + self.topic_loss_kl + self.config.reg * self.topic_loss_reg

        # define optimizer
        if self.config.opt == 'Adam':
            optimizer = tf.train.AdamOptimizer(self.config.lr)
        elif self.config.opt == 'Adagrad':
            optimizer = tf.train.AdagradOptimizer(self.config.lr)

        self.grad_vars = optimizer.compute_gradients(self.loss)
        self.clipped_grad_vars = [(tf.clip_by_value(grad, -self.config.grad_clip, self.config.grad_clip), var) for grad, var in self.grad_vars]
        self.opt = optimizer.apply_gradients(self.clipped_grad_vars, global_step=self.global_step)

        # growth criteria
#         self.dist_bow = tf.reduce_sum(tf.square(tf.expand_dims(tf.exp(logits_bow), 1) - tf.expand_dims(topic_bow, 0)), -1)
#         self.rads_bow = tf.sqrt(tf.reduce_sum(tf.multiply(prob_topic, self.dist_bow), 0) / tf.reduce_sum(prob_topic, 0))        
        self.dist_bow = -tf.matmul(self.t_variables['bow']/tf.reduce_sum(self.t_variables['bow'],1,keepdims=True), tf.log(topic_bow), transpose_b=True)
        self.rads_bow = tf.reduce_sum(tf.multiply(self.dist_bow, prob_topic), 0)
        
        # monitor
        n_bow = tf.reduce_sum(self.t_variables['bow'], 1)
        self.topic_ppls = tf.divide(self.topic_losses_recon, tf.maximum(1e-5, n_bow))
        self.topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices        
    
    def get_feed_dict(self, batch, mode='train'):
        bow = np.array([instance.bow for instance in batch]).astype(np.float32)
        keep_prob = self.config.keep_prob if mode == 'train' else 1.0
        feed_dict = {
                    self.t_variables['bow']: bow, 
                    self.t_variables['keep_prob']: keep_prob
        }
        return  feed_dict

# run

In [12]:
def get_loss(sess, batches, model):
    losses = []
    ppl_list = []
    rads_bow_list = []
    prob_topic_list = []
    for ct, batch in batches:
        feed_dict = model.get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, rads_bow_batch, prob_topic_batch \
            = sess.run([model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, model.rads_bow, tf.reduce_sum(model.prob_topic, 0)], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
        rads_bow_list.append(rads_bow_batch)
        prob_topic_list.append(prob_topic_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    rads_bow_mean = np.exp(np.sum(rads_bow_list, 0) / np.sum(prob_topic_list, 0))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, ppl_mean, rads_bow_mean

In [13]:
def print_topic_sample(tree_idxs, sess=None, model=None, rads_bow=None, parent_idx=0, topics_freq_bow_idxs=None, depth = 0):
    if topics_freq_bow_idxs is None:
        topics_freq_bow_idxs = bow_idxs[sess.run(model.topics_freq_bow_indices)]
        topic_freq_bow_idxs = topics_freq_bow_idxs[model.topic_idxs.index(parent_idx)]
        rad_bow = rads_bow[model.topic_idxs.index(parent_idx)]
        print(parent_idx, 'R: %.4f' % rad_bow, ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
    
    child_idxs = tree_idxs[parent_idx]
    depth += 1
    for child_idx in child_idxs:
        topic_freq_bow_idxs = topics_freq_bow_idxs[model.topic_idxs.index(child_idx)]
        rad_bow = rads_bow[model.topic_idxs.index(child_idx)]
        print('  '*depth, child_idx, 'R: %.4f' % rad_bow, ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        
        if child_idx in tree_idxs: print_topic_sample(tree_idxs, model=model, rads_bow=rads_bow, parent_idx=child_idx, topics_freq_bow_idxs=topics_freq_bow_idxs, depth=depth)


In [14]:
# def update_tree(rads_bow, model, threshold=0.1):
#     grow_topics = rads_bow > threshold
#     assert len(grow_topics) == len(model.topic_idxs)
#     update_tree_flg = False
#     for topic_idx, grow_topic in zip(model.topic_idxs, grow_topics):
#         if grow_topic and topic_idx != 0: # except for root node
#             update_tree_flg = True
#             parent_idx = model.child_to_parent_idxs[topic_idx]
#             if len(tree_idxs[parent_idx]) > 0:
#                 model.tree_idxs[parent_idx].append(max(tree_idxs[parent_idx])+1)
#             else:
#                 model.tree_idxs[parent_idx].append(10*parent_idx+1)
#     return tree_idxs, update_tree_flg

def add_descendant(topic_idx, tree_idxs, model):
    if topic_idx in tree_idxs:
        child_idx = max(tree_idxs[topic_idx])+1
        tree_idxs[topic_idx].append(child_idx)        
    else:
        child_idx = 10*topic_idx+1
        tree_idxs[topic_idx] = [10*topic_idx+1]
    return tree_idxs, child_idx

def update_tree(rads_bow, model, threshold=0.1):
    assert len(model.topic_idxs) == len(rads_bow)
    topic_rad_bow = {topic_idx: rad_bow for topic_idx, rad_bow in zip(model.topic_idxs, rads_bow)}
    update_tree_flg = False
    new_tree_idxs = copy.deepcopy(model.tree_idxs)
    for parent_idx, child_idxs in model.tree_idxs.items():
        rad_bow = np.sum([topic_rad_bow[child_idx] for child_idx in child_idxs])
        if rad_bow > threshold:
            update_tree_flg = True
            for depth in range(model.tree_depth[parent_idx], model.n_depth):
                new_tree_idxs, parent_idx = add_descendant(parent_idx, new_tree_idxs, model)
    return new_tree_idxs, update_tree_flg

In [15]:
losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','VALID:','TM','','',''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG']]))))

In [16]:
tree_idxs = {0:[1, 2], 
          1:[10, 11], 2:[20, 21]}

if 'sess' in globals(): sess.close()
model = Model(config, tree_idxs)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))}
saver = tf.train.Saver(max_to_keep=10)
update_tree_flg = False

In [17]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:    
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        # validate
#         if global_step_log % config.log_period == 0:
        if global_step_log % 5000 == 0:            
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, rads_bow_dev = get_loss(sess, dev_batches, model)

            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)

            # visualize topic
            print_topic_sample(tree_idxs, sess, model, rads_bow_dev)
            time_start = time.time()

            # update tree
            if global_step_log % 50000 == 0:
                tree_idxs, update_tree_flg = update_tree(rads_bow_dev, model, threshold=1000)
                if update_tree_flg:
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = Model(config, tree_idxs)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

display(log_df)
print_topic_sample()

TRAIN:   TM                      VALID:   TM          \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     40   10    9  111.58  489  110.39  1.06  0.13  103.86  456  102.54   
10000    39   20   19  110.95  466  109.54  1.33  0.08  103.46  436  101.92   
15000    39   30   29  110.66  455  109.13  1.47  0.06  103.52  434  101.90   
20000    39   40   39  110.49  448  108.89  1.55  0.05  103.50  432  101.83   
25000    40   50   49  110.38  443  108.73  1.61  0.04  103.40  428  101.69   
30000    39   60   59  110.29  440  108.61  1.65  0.04  103.47  430  101.76   
35000    39   70   69  110.23  438  108.52  1.68  0.04  103.40  427  101.67   
40000    39   80   79  110.18  435  108.44  1.71  0.03  103.35  425  101.61   
45000    43   90   89  110.14  434  108.38  1.73  0.03  103.20  419  101.46   
50000    41  100   99  110.10  432  108.33  1.75  0.03  103.16  419  101.39   
55000    59  110  109  110.09  432  108.30  1.76  0.03  103.32  419  101.44   
60000    63  120  119  110.07  431  108.25  1.78  0.04  103.20  416  101.30   
65000    54  130  129  110.05  429  108.21  1.80  0.04  103.07  413  101.14   
70000    54  140  139  110.03  428  108.17  1.82  0.04  103.04  411  101.10   
75000    57  150  149  110.00  427  108.13  1.84  0.03  103.22  415  101.24   
80000    55  160  159  109.98  426  108.09  1.85  0.03  103.05  414  101.07   
85000    54  170  169  109.96  426  108.06  1.87  0.03  103.04  409  101.01   
90000    55  180  179  109.94  425  108.03  1.89  0.03  103.00  408  100.96   
95000    54  190  189  109.92  424  108.00  1.90  0.03  103.10  411  101.05   
100000   54  200  199  109.91  423  107.97  1.91  0.03  103.06  409  101.00   
105000   77  210  209  109.89  422  107.94  1.93  0.03  102.88  403  100.75   
110000   71  220  219  109.88  422  107.91  1.94  0.03  103.02  408  100.88   
115000   74  230  229  109.86  421  107.88  1.95  0.03  102.85  404  100.68   
120000   80  240  239  109.85  420  107.85  1.97  0.03  103.03  406  100.86   
125000   83  250  249  109.83  419  107.82  1.98  0.03  102.93  404  100.74   
130000   72  260  259  109.81  419  107.79  1.99  0.03  102.83  400  100.61   
135000   84  270  269  109.80  418  107.76  2.00  0.03  102.88  399  100.66   
140000   72  280  279  109.78  417  107.73  2.02  0.03  102.62  393  100.37   

                    
          KL   REG  
5000    1.29  0.03  
10000   1.52  0.02  
15000   1.60  0.02  
20000   1.65  0.02  
25000   1.70  0.02  
30000   1.70  0.01  
35000   1.72  0.02  
40000   1.72  0.02  
45000   1.73  0.02  
50000   1.75  0.02  
55000   1.81  0.07  
60000   1.85  0.05  
65000   1.89  0.03  
70000   1.92  0.02  
75000   1.96  0.02  
80000   1.97  0.02  
85000   2.02  0.02  
90000   2.02  0.02  
95000   2.03  0.02  
100000  2.05  0.02  
105000  2.08  0.05  
110000  2.10  0.04  
115000  2.13  0.04  
120000  2.13  0.04  
125000  2.16  0.04  
130000  2.19  0.04  
135000  2.19  0.04  
140000  2.22  0.04

0 R: 1381.9486 ; & - : nice price made bit big material
   1 R: 837.6700 carry pockets back room comfortable compartments work lot space stuff
     10 R: 844.2677 pocket room ipad carry small mouse power charger netbook inside
     11 R: 2377.6572 love perfect recommend price perfectly highly pro bought mac cute
     12 R: 1685.6971 ! bought quality ... size buy purchased gift loves purse
     13 R: 5197.4653 ! bought loves ... awesome buy love gift absolutely daughter
   2 R: 4361.1011 ... quality $ amazon leather reviews price ? item high
     20 R: 1496.0200 sleeve protection air pro inside smell inch protect snug neoprene
     21 R: 1421.5476 cover keyboard mac hard pro nice apple easy perfectly color
     22 R: 3468.9080 color picture pink blue ordered wanted bought perfectly happy bright
     23 R: 2118.1453 color picture ordered pink blue received shipping fast arrived bright
   3 R: 1606.4722 strap handle zipper zippers years shoulder straps back stitching 've
     31 R: 1340.6

KeyboardInterrupt: 

# confirm variables

In [ ]:
states_topic_embeddings = tf.concat([tree_states_topic_embeddings[topic_idx] for topic_idx in topic_idxs], 0)

In [ ]:
debug_value([states_topic_embeddings[:, :6]])

In [ ]:
topic_embeddings = tf.concat([tree_topic_embeddings[topic_idx] for topic_idx in topic_idxs], 0)

In [ ]:
debug_value([topic_embeddings[:, :6]])

In [ ]:
_topics_bow, = debug_value([topic_bow], return_value=True)
np.max(_topics_bow)

In [ ]:
_topics_bow, = debug_value([topic_bow], return_value=True)

plt.figure(figsize=(12, 20))
    
_topic_bow = _topics_bow[0]
plt.subplot(5,3,2)
plt.ylim([0, 0.1])
plt.bar(bow_idxs, _topic_bow)

for i in range(1, len(topic_idxs)):
    _topic_bow = _topics_bow[i]
    plt.subplot(5,3,i+3)
    plt.ylim([0, 0.1])
#     plt.axis('off')
    plt.bar(bow_idxs, _topic_bow)

plt.show()

In [ ]:
_prob_topics = []
for ct, batch in dev_batches:
    feed_dict = get_feed_dict(batch)
    _prob_topic, = sess.run([prob_topic], feed_dict = feed_dict)
    _prob_topics.append(_prob_topic)
    
_prob_topics = np.concatenate(_prob_topics, 0)
_prob_topic_mean = np.mean(_prob_topics, 0)

print(_prob_topic_mean)

In [ ]:
debug_value([topic_dots])

In [ ]:
debug_value([topic_losses_reg])

In [ ]:
tree_mask_reg

In [ ]:
_topic_bow, = debug_value([topic_bow], return_value=True)

In [ ]:
plt.bar(bow_idxs, _topic_bow[0])

In [ ]:
plt.bar(bow_idxs, _topic_bow[1])

In [ ]:
plt.bar(bow_idxs, _topic_bow[2])

In [ ]:
plt.bar(bow_idxs, _topic_bow[3])

In [ ]:
np.max(_topic_bow, 1)

In [ ]:
plt.bar(bow_idxs, _topic_bow[-5])

In [ ]:
plt.bar(bow_idxs, _topic_bow[-1])

In [ ]:
len(bow_idxs)

In [ ]:
debug_value([prob_topic[3]])

In [ ]:
debug_value([tf.exp(-tf.divide(topic_losses_recon, n_bow))])

### test

In [ ]:
debug_shape([bow, hidden_bow, latents_bow, prob_topic, bow_embeddings, topic_embeddings, topic_bow, prob_bow])

In [ ]:
debug_shape([topic_losses_recon, topic_loss_recon, n_bow, ppls, topic_embeddings_norm, tf.expand_dims(topic_angles_mean, -1), topic_angles_vars])

In [ ]:
debug_value([tf.reduce_sum(tf.square(topic_embeddings_norm), 1)], return_value=True)[0]

In [ ]:
debug_value([tf.reduce_sum(prob_topic, -1), tf.reduce_sum(topic_bow, -1), tf.reduce_sum(tf.exp(prob_bow), 1)])

In [ ]:
sigma_bow = tf.exp(0.5 * logvars_bow)
dist_bow = tfd.Normal(means_bow, sigma_bow)
dist_std = tfd.Normal(0., 1.)
topic_loss_kl_tmp = tf.reduce_mean(tf.reduce_sum(tfd.kl_divergence(dist_bow, dist_std), 1))

In [ ]:
debug_value([topic_loss_recon, topic_loss_kl, topic_loss_kl_tmp])

In [ ]:
_logvars, _means, _kl_losses, _latents, _output_logits = sess.run([logvars, means, kl_losses, latents, output_logits], feed_dict=feed_dict)


In [ ]:
_logvars.shape, _means.shape, _kl_losses.shape, _latents.shape

In [ ]:
_output_logits

In [ ]:
_output_logits, _dec_target_idxs_do, _dec_mask_tokens_do, _recon_loss, _kl_losses, _ = sess.run([output_logits, dec_target_idxs_do, dec_mask_tokens_do, recon_loss, kl_losses, opt], feed_dict=feed_dict)


In [ ]:
tf.reduce_max(output_logits, 2).eval(session=sess, feed_dict=feed_dict).shape

In [ ]:
_output_logits.shape, _dec_target_idxs_do.shape, _dec_mask_tokens_do.shape

In [ ]:
_logits = np.exp(_output_logits) / np.sum(np.exp(_output_logits), 2)[:, :, None]

In [ ]:
_idxs = _dec_target_idxs_do

In [ ]:
_losses = np.array([[-np.log(_logits[i, j, _idxs[i, j]]) for j in range(_idxs.shape[1])] for i in range(_idxs.shape[0])]) * _dec_mask_tokens_do

In [ ]:
np.sum(_losses)/np.sum(_dec_mask_tokens_do)

In [ ]:
_recon_loss

In [ ]:
_kl_losses.shape